<a href="https://colab.research.google.com/github/bahaeddine-toumi/Email-Classification-/blob/main/Sentiment_Analyses.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_excel('ALL.xlsx')

In [ ]:
df

,Body,Class_Sentiment
0,Attached is a revised Credit Watch listing for...,neg
1,Continental has a price of $297.17(non-refunda...,pos
2,You hurt my feelings bc you didn't wait for my...,neg
3,Attached is the final notification report for ...,pos
4,The basic reason we are changing meters in Si...,pos
...,...,...
93843,"Sergey, itâ€™s working.Â Thanks. Feel free to...",pos
93844,"Hi Dmytrii,\n\nThanks for the links related to...",neu
93845,"Hi Daryna,\n\n \n\nNo worries â€“ if we stay w...",neu
93846,"John, can you send some screen shots of what y...",neu


In [ ]:
df['Class_Sentiment'] = pd.factorize(df['Class_Sentiment'])[0]

In [ ]:
df

,Body,Class_Sentiment
0,Attached is a revised Credit Watch listing for...,0
1,Continental has a price of $297.17(non-refunda...,1
2,You hurt my feelings bc you didn't wait for my...,0
3,Attached is the final notification report for ...,1
4,The basic reason we are changing meters in Si...,1
...,...,...
93843,"Sergey, itâ€™s working.Â Thanks. Feel free to...",1
93844,"Hi Dmytrii,\n\nThanks for the links related to...",2
93845,"Hi Daryna,\n\n \n\nNo worries â€“ if we stay w...",2
93846,"John, can you send some screen shots of what y...",2


In [ ]:
import re
import nltk

nltk.download('stopwords')

from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

all_stopwords = stopwords.words('english')
all_stopwords.remove('not')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
corpus=[]
label=[]

for i in range(len(df['Body'])):
  review = re.sub('[^a-zA-Z]', ' ', df['Body'][i])
  review = review.lower()
  review = review.split()
  review = [ps.stem(word) for word in review if not word in set(all_stopwords)]
  review = ' '.join(review)
  label.append(df['Class_Sentiment'][i])
  corpus.append(review)

In [ ]:
len(label)

93848

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features = 1420)

In [ ]:
X = cv.fit_transform(corpus).toarray()
y = df.iloc[:, -1].values

In [ ]:
X.size

133264160

In [ ]:
# Saving BoW dictionary to later use in prediction
import pickle
bow_path = 'c1_BoW_Sentiment_Model.pkl'
pickle.dump(cv, open(bow_path, "wb"))

In [ ]:
y=y.reshape((93848,1))

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 0)

In [ ]:
from sklearn.naive_bayes import GaussianNB
classifier = GaussianNB()
classifier.fit(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GaussianNB()

In [ ]:
# Exporting NB Classifier to later use in prediction
import joblib
joblib.dump(classifier, 'c2_Classifier_Sentiment_Model') 

['c2_Classifier_Sentiment_Model']

In [ ]:
y_pred = classifier.predict(X_test)

from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)

accuracy_score(y_test, y_pred)

[[9585  230  440]
 [2096 2264 3473]
 [ 206   75  401]]


0.6526371870005327

In [ ]:
dataset = pd.read_csv('a2_RestaurantReviews_FreshDump.tsv', delimiter = '\t', quoting = 3)
dataset.head()

,Review
0,Spend your money elsewhere.
1,Their regular toasted bread was equally satisf...
2,The Buffet at Bellagio was far from what I ant...
3,"And the drinks are WEAK, people!"
4,-My order was not correct.


In [ ]:
dataset[99]="i can't help you."

In [ ]:
import re
import nltk

nltk.download('stopwords')

from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

all_stopwords = stopwords.words('english')
all_stopwords.remove('not')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
corpus=[]

for i in range(0, 100):
  review = re.sub('[^a-zA-Z]', ' ', dataset['Review'][i])
  review = review.lower()
  review = review.split()
  review = [ps.stem(word) for word in review if not word in set(all_stopwords)]
  review = ' '.join(review)
  corpus.append(review)

In [ ]:
# Loading BoW dictionary
from sklearn.feature_extraction.text import CountVectorizer
import pickle
cvFile='c1_BoW_Sentiment_Model.pkl'
# cv = CountVectorizer(decode_error="replace", vocabulary=pickle.load(open('./drive/MyDrive/Colab Notebooks/2 Sentiment Analysis (Basic)/3.1 BoW_Sentiment Model.pkl', "rb")))
cv = pickle.load(open(cvFile, "rb"))

In [ ]:
X_fresh = cv.transform(corpus).toarray()
X_fresh.shape

(100, 1420)

In [ ]:
import joblib
classifier = joblib.load('c2_Classifier_Sentiment_Model')

In [ ]:
y_pred = classifier.predict(X_fresh)
print(y_pred)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 0 0
 0 0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 2 0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
